In [1]:
import pandas as pd
import numpy as np
import pymysql
from ast import literal_eval
import warnings; warnings.simplefilter('ignore')

In [2]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='dv_db',
)

In [3]:
sql = "SELECT userId, movieId, rating FROM ratings"

with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
    
ratings = pd.DataFrame(result, columns =['userId', 'movieId', 'rating'])

In [4]:
sql = "SELECT id, movieId, genres, cast, keywords, director, title, vote_count, vote_average, year FROM movies"

with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()

In [5]:
connection.close()

In [6]:
movies = pd.DataFrame(result, columns =['id', 'movieId', 'genres', 'cast', 'keywords', 'director', 'title', 'vote_count', 'vote_average', 'year'])

In [7]:
#Hybrid wala thing

In [8]:
movies['cast'] = movies['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [9]:
movies['director'] = movies['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [10]:
id_map = movies[['movieId', 'id']]
id_map = id_map.merge(movies[['title', 'id']], on='id').set_index('title')

In [11]:
id_map

,movieId,id
title,,
Toy Story,1,862
Jumanji,2,8844
Grumpier Old Men,3,15602
Waiting to Exhale,4,31357
Father of the Bride Part II,5,11862
...,...,...
The Last Brickmaker in America,161944,159550
Rustom,162542,392572
Mohenjo Daro,162672,402672


In [12]:
indices_map = id_map.set_index('id')

In [13]:
movies = movies.reset_index()
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

In [14]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer

In [16]:
s = movies.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s = s[s > 1]

In [17]:
s

                                                                                                                             783
independent film                                                                                                              92
woman director                                                                                                                62
musical                                                                                                                       20
duringcreditsstinger                                                                                                          19
                                                                                                                            ... 
independent film,duringcreditsstinger                                                                                          2
world war ii                                                                                     

In [18]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [19]:
stemmer = SnowballStemmer('english')

In [20]:
movies['keywords'] = movies['keywords'].apply(filter_keywords)
movies['keywords'] = movies['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [21]:
movies['director'] = movies['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
movies['director'] = movies['director'].apply(lambda x: [x,x, x])

In [22]:
movies['genres'] = movies['genres'].apply(lambda x: x.split(','))

In [23]:
movies['soup'] = movies['keywords'] + movies['cast'] + movies['director'] + movies['genres']
movies['soup'] = movies['soup'].apply(lambda x: ' '.join(x))

In [24]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(movies['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [25]:
print(count.get_feature_names())

['aaronblaise', 'aaronblaise aaronblaise', 'aaronblaise adventure', 'aaronharvey', 'aaronharvey aaronharvey', 'aaronharvey drama', 'aaronnorris', 'aaronnorris aaronnorris', 'aaronnorris action', 'abbaskiarostami', 'abbaskiarostami abbaskiarostami', 'abbaskiarostami drama', 'abbewool', 'abbewool abbewool', 'abbewool action', 'abbyepstein', 'abbyepstein abbyepstein', 'abbyepstein documentary', 'abdellatifkechiche', 'abdellatifkechiche abdellatifkechiche', 'abdellatifkechiche romance', 'abelevitow', 'abelevitow abelevitow', 'abelevitow fantasy', 'abelferrara', 'abelferrara abelferrara', 'abelferrara crime', 'abelferrara drama', 'abelferrara horror', 'abelferrara thriller', 'abelgance', 'abelgance abelgance', 'abelgance drama', 'abrahampolonsky', 'abrahampolonsky abrahampolonsky', 'abrahampolonsky drama', 'abrams', 'abrams abrams', 'abrams action', 'abrams adventure', 'abrams science', 'abrams thriller', 'action', 'action adventure', 'action animation', 'action comedy', 'action crime', 'ac

In [26]:
print(count_matrix.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [26]:
print(cosine_sim)

[[1.         0.05263158 0.05564149 ... 0.         0.         0.        ]
 [0.05263158 1.         0.         ... 0.05006262 0.04588315 0.        ]
 [0.05564149 0.         1.         ... 0.05292561 0.         0.        ]
 ...
 [0.         0.05006262 0.05292561 ... 1.         0.13093073 0.        ]
 [0.         0.04588315 0.         ... 0.13093073 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [27]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
svd = SVD()

In [28]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [29]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [30]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:100]
    movie_indices = [i[0] for i in sim_scores]
    
    recommendedMovies = movies.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    recommendedMovies['est'] = recommendedMovies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    recommendedMovies = recommendedMovies.sort_values('est', ascending=False)
    return recommendedMovies

In [31]:
hybrid(1, 'Avatar').head(10)

,title,vote_count,vote_average,year,id,est
1011,The Terminator,4208.0,7.40000,1984,218,3.207109
974,Aliens,3282.0,7.70000,1986,679,3.123804
6179,A Trip to the Moon,315.0,7.90667,1902,775,3.122812
8626,Captain America: The Winter Soldier,5882.0,7.60041,2014,100402,3.112057
1131,Star Trek II: The Wrath of Khan,688.0,7.30000,1982,154,3.058949
344,True Lies,1138.0,6.80000,1994,36955,3.054220
7969,The Avengers,12001.0,7.40022,2012,24428,3.053823
8658,X-Men: Days of Future Past,6155.0,7.50000,2014,127585,3.039638
8868,Avengers: Age of Ultron,6909.0,7.30039,2015,99861,3.015966
8869,Ant-Man,6030.0,7.00050,2015,102899,3.010365
